# Creates specific joins associated with Sanitary Sewer System


## Setup

### Imports and login

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, Table, FeatureLayerCollection
import json

In [ ]:
gis = GIS("home")

### Safety Checks

In [ ]:
safety_check = "No"  # Safety Check
awareness_check = "No"  # awareness check

In [ ]:
# Convert the input to lowercase and check if it is "yes"
if safety_check.strip().lower() != "yes":
    raise ValueError("Safety check failed. The script will stop because safety_check is not 'Yes'.")

In [ ]:
# Convert the input to lowercase and check if it is "yes"
if awareness_check.strip().lower() != "yes":
    raise ValueError("Safety check failed. The script will stop because safety_check is not 'Yes'.")

### Functions

In [ ]:
def get_service_name_from_url(url):
    """
    Given a URL like:
    .../rest/services/<serviceName>/FeatureServer/<layerId>
    returns <serviceName>
    """
    parts = url.split('/')
    try:
        index = parts.index('services')
        service_name = parts[index+1]
    except Exception:
        service_name = None
    return service_name

In [ ]:
def update_dashboards_replace_marker(gis, view_name, new_itemid):
    """
    Searches all dashboards for the marker "old_<view_name>_itemID_here" and
    replaces it with the new_itemid.
    """
    marker = f"old_{view_name}_itemID_here"
    dashboards = gis.content.search(query='type:"Dashboard"', max_items=100)
    print(f"Searching for marker '{marker}' in {len(dashboards)} dashboards...")
    
    for dash in dashboards:
        data = dash.get_data()
        modified = False
        if data and "arcadeDataSourceItems" in data:
            for arcade_item in data["arcadeDataSourceItems"]:
                if "script" in arcade_item:
                    script = arcade_item["script"]
                    if marker in script:
                        new_script = script.replace(marker, new_itemid)
                        arcade_item["script"] = new_script
                        modified = True
                        print(f"Updated dashboard '{dash.title}' (ID: {dash.id}) replacing marker '{marker}' with '{new_itemid}'.")
            if modified:
                dash.update(data=data)
                print(f"Dashboard '{dash.title}' updated.")
        else:
            print(f"Dashboard '{dash.title}' (ID: {dash.id}) has no arcadeDataSourceItems; skipping.")


## ssMain_CCTVInspVW

### Initial Service

In [ ]:
# join view name
view_name = "ssMain_CCTVInspVW"

In [ ]:
# Define URLs for the gravity main layer (Sanitary Sewer, layer 16)
# and the CCTV table (layer 2)
gravity_layer_url = "https://services3.arcgis.com/X0xdaFqVSAx896l1/arcgis/rest/services/SanitarySewer/FeatureServer/16"
cctv_table_url = "https://services3.arcgis.com/X0xdaFqVSAx896l1/arcgis/rest/services/Sanitary_CCTV_Pipe_Conditions/FeatureServer/2"

In [ ]:
gravity_fl = FeatureLayer(gravity_layer_url, gis)
cctv_tbl = Table(cctv_table_url, gis)

# Retrieve the spatial reference from the gravity main layer
gravity_sr = gravity_fl.properties.extent.spatialReference
wkid = gravity_sr.get("wkid", 102100)  # default to 102100 if not found
# print("Using spatial reference WKID:", wkid)

In [ ]:
# Create the view service with the correct spatial reference
view_service = gis.content.create_service(
    name=view_name,
    is_view=True,
    wkid=wkid
)
view_flc = FeatureLayerCollection.fromitem(view_service)

### To Add to Service

#### Fields

In [ ]:
# Define fields for the gravity main layer (Sanitary Sewer)
sourceFeatureLayerFields = [
    {"name": "OBJECTID", "alias": "OBJECTID", "source": "OBJECTID"},
    {"name": "FACILITYID", "alias": "Facility Identifier", "source": "FACILITYID"},
    {"name": "INSTALLDATE", "alias": "Install Date", "source": "INSTALLDATE"},
    {"name": "MATERIAL", "alias": "Material", "source": "MATERIAL"},
    {"name": "DIAMETER", "alias": "Diameter", "source": "DIAMETER"},
    {"name": "MAINSHAPE", "alias": "Main Shape", "source": "MAINSHAPE"},
    {"name": "LINEDYEAR", "alias": "Year Lined", "source": "LINEDYEAR"},
    {"name": "LINERTYPE", "alias": "Liner Type", "source": "LINERTYPE"},
    {"name": "FROMMH", "alias": "From Manhole", "source": "FROMMH"},
    {"name": "TOMH", "alias": "To Manhole", "source": "TOMH"},
    {"name": "WATERTYPE", "alias": "Water Type", "source": "WATERTYPE"},
    {"name": "ENABLED", "alias": "Enabled", "source": "ENABLED"},
    {"name": "ACTIVEFLAG", "alias": "Active Flag", "source": "ACTIVEFLAG"},
    {"name": "OWNEDBY", "alias": "Owned By", "source": "OWNEDBY"},
    {"name": "MAINTBY", "alias": "Managed By", "source": "MAINTBY"},
    {"name": "SUMFLOW", "alias": "Flow Summary", "source": "SUMFLOW"},
    {"name": "LASTUPDATE", "alias": "Last Update Date", "source": "LASTUPDATE"},
    {"name": "LASTEDITOR", "alias": "Last Editor", "source": "LASTEDITOR"},
    {"name": "DOWNELEV", "alias": "Downstream Elevation", "source": "DOWNELEV"},
    {"name": "UPELEV", "alias": "Upstream Elevation", "source": "UPELEV"},
    {"name": "SLOPE", "alias": "Slope", "source": "SLOPE"},
    {"name": "LifeCycleStatus", "alias": "Life Cycle Status", "source": "LifeCycleStatus"},
    {"name": "Notes", "alias": "Notes", "source": "Notes"},
    {"name": "RecordLength", "alias": "Record Length", "source": "RecordLength"},
    {"name": "RecordDrawing", "alias": "Record Drawing", "source": "RecordDrawing"},
    {"name": "Shape__Length", "alias": "Shape__Length", "source": "Shape__Length"},
    {"name": "CreationDate", "alias": "CreationDate", "source": "CreationDate"},
    {"name": "Creator", "alias": "Creator", "source": "Creator"},
    {"name": "EditDate", "alias": "EditDate", "source": "EditDate"},
    {"name": "Editor", "alias": "Editor", "source": "Editor"},
    {"name": "GlobalID", "alias": "GlobalID", "source": "GlobalID"},
    {"name": "PipeType", "alias": "Pipe Sub Type", "source": "PipeType"}
]

In [ ]:
# Define fields for the CCTV table
sourceTableFields = [
    {"name": "OBJECTID", "alias": "OBJECTID", "source": "OBJECTID"},
    {"name": "InspectionID", "alias": "InspectionID", "source": "InspectionID"},
    {"name": "Surveyed_By", "alias": "Surveyed_By", "source": "Surveyed_By"},
    {"name": "Certificate_Number", "alias": "Certificate_Number", "source": "Certificate_Number"},
    {"name": "Reviewed_By", "alias": "Reviewed_By", "source": "Reviewed_By"},
    {"name": "Reviewer_Certificate_Number", "alias": "Reviewer_Certificate_Number", "source": "Reviewer_Certificate_Number"},
    {"name": "Owner", "alias": "Owner", "source": "Owner"},
    {"name": "Customer", "alias": "Customer", "source": "Customer"},
    {"name": "PO_Number", "alias": "PO_Number", "source": "PO_Number"},
    {"name": "WorkOrder", "alias": "WorkOrder", "source": "WorkOrder"},
    {"name": "Media_Label", "alias": "Media_Label", "source": "Media_Label"},
    {"name": "Project", "alias": "Project", "source": "Project"},
    {"name": "Inspection_Time", "alias": "Inspection_Time", "source": "Inspection_Time"},
    {"name": "Sheet_Number", "alias": "Sheet_Number", "source": "Sheet_Number"},
    {"name": "Weather", "alias": "Weather", "source": "Weather"},
    {"name": "PreCleaning", "alias": "PreCleaning", "source": "PreCleaning"},
    {"name": "Flow_Control", "alias": "Flow_Control", "source": "Flow_Control"},
    {"name": "Purpose", "alias": "Purpose", "source": "Purpose"},
    {"name": "Direction", "alias": "Direction", "source": "Direction"},
    {"name": "Inspection_Technology_Used_CCTV", "alias": "Inspection_Technology_Used_CCTV", "source": "Inspection_Technology_Used_CCTV"},
    {"name": "Inspection_Technology_Used_Lase", "alias": "Inspection_Technology_Used_Laser", "source": "Inspection_Technology_Used_Lase"},
    {"name": "Inspection_Technology_Used_Sona", "alias": "Inspection_Technology_Used_Sonar", "source": "Inspection_Technology_Used_Sona"},
    {"name": "Inspection_Technology_Used_Side", "alias": "Inspection_Technology_Used_Sidewall", "source": "Inspection_Technology_Used_Side"},
    {"name": "Inspection_Technology_Used_Zoom", "alias": "Inspection_Technology_Used_Zoom", "source": "Inspection_Technology_Used_Zoom"},
    {"name": "Inspection_Technology_Used_Othe", "alias": "Inspection_Technology_Used_Other", "source": "Inspection_Technology_Used_Othe"},
    {"name": "Inspection_Status", "alias": "Inspection_Status", "source": "Inspection_Status"},
    {"name": "Consequence_Of_Failure", "alias": "Consequence_Of_Failure", "source": "Consequence_Of_Failure"},
    {"name": "Pressure_Value", "alias": "Pressure_Value", "source": "Pressure_Value"},
    {"name": "Drainage_Area", "alias": "Drainage_Area", "source": "Drainage_Area"},
    {"name": "Pipe_Segment_Reference", "alias": "Pipe_Segment_Reference", "source": "Pipe_Segment_Reference"},
    {"name": "Street", "alias": "Street", "source": "Street"},
    {"name": "City", "alias": "City", "source": "City"},
    {"name": "Location_Code", "alias": "Location_Code", "source": "Location_Code"},
    {"name": "Location_Details", "alias": "Location_Details", "source": "Location_Details"},
    {"name": "Pipe_Use", "alias": "Pipe_Use", "source": "Pipe_Use"},
    {"name": "Height", "alias": "Height", "source": "Height"},
    {"name": "Width", "alias": "Width", "source": "Width"},
    {"name": "Material", "alias": "Material", "source": "Material"},
    {"name": "Lining_Method", "alias": "Lining_Method", "source": "Lining_Method"},
    {"name": "Coating_Method", "alias": "Coating_Method", "source": "Coating_Method"},
    {"name": "Pipe_Joint_Length", "alias": "Pipe_Joint_Length", "source": "Pipe_Joint_Length"},
    {"name": "Total_Length", "alias": "Total_Length", "source": "Total_Length"},
    {"name": "Length_Surveyed", "alias": "Length_Surveyed", "source": "Length_Surveyed"},
    {"name": "Year_Constructed", "alias": "Year_Constructed", "source": "Year_Constructed"},
    {"name": "Year_Renewed", "alias": "Year_Renewed", "source": "Year_Renewed"},
    {"name": "Upstream_MH", "alias": "Upstream_MH", "source": "Upstream_MH"},
    {"name": "Up_Rim_to_Invert", "alias": "Up_Rim_to_Invert", "source": "Up_Rim_to_Invert"},
    {"name": "Up_Grade_to_Invert", "alias": "Up_Grade_to_Invert", "source": "Up_Grade_to_Invert"},
    {"name": "Up_Rim_to_Grade", "alias": "Up_Rim_to_Grade", "source": "Up_Rim_to_Grade"},
    {"name": "Up_Northing", "alias": "Up_Northing", "source": "Up_Northing"},
    {"name": "Up_Easting", "alias": "Up_Easting", "source": "Up_Easting"},
    {"name": "Up_Elevation", "alias": "Up_Elevation", "source": "Up_Elevation"},
    {"name": "Downstream_MH", "alias": "Downstream_MH", "source": "Downstream_MH"},
    {"name": "Down_Rim_to_Invert", "alias": "Down_Rim_to_Invert", "source": "Down_Rim_to_Invert"},
    {"name": "Down_Grade_to_Invert", "alias": "Down_Grade_to_Invert", "source": "Down_Grade_to_Invert"},
    {"name": "Down_Rim_to_Grade", "alias": "Down_Rim_to_Grade", "source": "Down_Rim_to_Grade"},
    {"name": "Down_Northing", "alias": "Down_Northing", "source": "Down_Northing"},
    {"name": "Down_Easting", "alias": "Down_Easting", "source": "Down_Easting"},
    {"name": "Down_Elevation", "alias": "Down_Elevation", "source": "Down_Elevation"},
    {"name": "MH_Coordinate_System", "alias": "MH_Coordinate_System", "source": "MH_Coordinate_System"},
    {"name": "Vertical_Datum", "alias": "Vertical_Datum", "source": "Vertical_Datum"},
    {"name": "GPS_Accuracy", "alias": "GPS_Accuracy", "source": "GPS_Accuracy"},
    {"name": "Additional_Info", "alias": "Additional_Info", "source": "Additional_Info"},
    {"name": "Reverse_Setup", "alias": "Reverse_Setup", "source": "Reverse_Setup"},
    {"name": "IsImperial", "alias": "IsImperial", "source": "IsImperial"},
    {"name": "OBJECTID_1", "alias": "OBJECTID", "source": "OBJECTID_1"},
    {"name": "RatingID", "alias": "RatingID", "source": "RatingID"},
    {"name": "InspectionID_1", "alias": "InspectionID", "source": "InspectionID_1"},
    {"name": "STGradeScore1", "alias": "STGradeScore1", "source": "STGradeScore1"},
    {"name": "STGradeScore2", "alias": "STGradeScore2", "source": "STGradeScore2"},
    {"name": "STGradeScore3", "alias": "STGradeScore3", "source": "STGradeScore3"},
    {"name": "STGradeScore4", "alias": "STGradeScore4", "source": "STGradeScore4"},
    {"name": "STGradeScore5", "alias": "STGradeScore5", "source": "STGradeScore5"},
    {"name": "OMGradeScore1", "alias": "OMGradeScore1", "source": "OMGradeScore1"},
    {"name": "OMGradeScore2", "alias": "OMGradeScore2", "source": "OMGradeScore2"},
    {"name": "OMGradeScore3", "alias": "OMGradeScore3", "source": "OMGradeScore3"},
    {"name": "OMGradeScore4", "alias": "OMGradeScore4", "source": "OMGradeScore4"},
    {"name": "OMGradeScore5", "alias": "OMGradeScore5", "source": "OMGradeScore5"},
    {"name": "STPipeRating", "alias": "STPipeRating", "source": "STPipeRating"},
    {"name": "OMPipeRating", "alias": "OMPipeRating", "source": "OMPipeRating"},
    {"name": "OverallPipeRating", "alias": "OverallPipeRating", "source": "OverallPipeRating"},
    {"name": "STQuickRating", "alias": "STQuickRating", "source": "STQuickRating"},
    {"name": "OMQuickRating", "alias": "OMQuickRating", "source": "OMQuickRating"},
    {"name": "PACPQuickRating", "alias": "PACPQuickRating", "source": "PACPQuickRating"},
    {"name": "STPipeRatingsIndex", "alias": "STPipeRatingsIndex", "source": "STPipeRatingsIndex"},
    {"name": "OMPipeRatingsIndex", "alias": "OMPipeRatingsIndex", "source": "OMPipeRatingsIndex"},
    {"name": "OverallPipeRatingsIndex", "alias": "OverallPipeRatingsIndex", "source": "OverallPipeRatingsIndex"},
    {"name": "LoFPACP", "alias": "LoFPACP", "source": "LoFPACP"},
    {"name": "Risk", "alias": "Risk", "source": "Risk"},
    {"name": "PipeShape", "alias": "PipeShape", "source": "PipeShape"},
    {"name": "Inspection_Date", "alias": "Inspection_Date", "source": "Inspection_Date"},
    {"name": "Date_Cleaned", "alias": "Date_Cleaned", "source": "Date_Cleaned"},
    {"name": "VideoLink", "alias": "Video Link", "source": "VideoLink"},
    {"name": "VideoLink2", "alias": "Second Video Link", "source": "VideoLink2"},
    {"name": "ACINotes", "alias": "Abonmarche Notes", "source": "ACINotes"},
    {"name": "CreationDate", "alias": "CreationDate", "source": "CreationDate"},
    {"name": "Creator", "alias": "Creator", "source": "Creator"},
    {"name": "EditDate", "alias": "EditDate", "source": "EditDate"},
    {"name": "Editor", "alias": "Editor", "source": "Editor"}
]

#### Definition

In [ ]:
# Specify the join fields: join gravity layer's FACILITYID to the CCTV table's Pipe_Segment_Reference
field_to_join_fl = "FACILITYID"
field_to_join_tbl = "Pipe_Segment_Reference"

In [ ]:
# Extract the service names from the URLs
gravity_service_name = get_service_name_from_url(gravity_layer_url)
cctv_service_name = get_service_name_from_url(cctv_table_url)

In [ ]:
# Build the view definition with the join configuration
definition_to_add = {
    "layers": [
        {
            "name": view_name,
            "displayField": "FACILITYID",
            "description": "AttributeJoin",
            "adminLayerInfo": {
                "viewLayerDefinition": {
                    "table": {
                        "name": "Target_fl",
                        "sourceServiceName": gravity_service_name,
                        "sourceLayerId": 16,
                        "sourceLayerFields": sourceFeatureLayerFields,
                        "relatedTables": [
                            {
                                "name": "JoinedTable",
                                "sourceServiceName": cctv_service_name,
                                "sourceLayerId": 2,
                                "sourceLayerFields": sourceTableFields,
                                "type": "INNER",
                                "parentKeyFields": [field_to_join_fl],
                                "keyFields": [field_to_join_tbl],
                                "topFilter": {
                                    "groupByFields": field_to_join_tbl,
                                    "orderByFields": "Inspection_Date DESC",
                                    "topCount": 1
                                }
                            }
                        ],
                        "materialized": False
                    }
                },
                "geometryField": {
                    "name": f"{gravity_service_name}.Shape"
                }
            }
        }
    ]
}


### Execute

In [ ]:
# Apply the join view definition
view_flc.manager.add_to_definition(definition_to_add)
print(f"View layer {view_name} created successfully.")

In [ ]:
# Update dashboards
new_itemid = view_service.id
print("New join view item id:", new_itemid)
update_dashboards_replace_marker(gis, view_name, new_itemid)

## Second Join View

### Initial Service

### To Add to Service

#### Fields

#### Definition

### Execute

## Third Join View

### Initial Service

### To Add to Service

#### Fields

#### Definition

### Execute